for this task, the first and quick idea brings to my mind was a binary classification task. First, I extracted research problem sentences from research-problem.json, and using a similarity function, I labeled the dataset with 0 and 1 labels to represent "non research problem" and "research problem" respectively. in he following cells, I will explain more about the dataset creation.


In [17]:
!git clone https://github.com/ncg-task/test-data.git

fatal: destination path 'test-data' already exists and is not an empty directory.


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.4 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 101 kB 7.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# 
import json
import os
import glob
import tensorflow as tf
from transformers import BertConfig, BertTokenizerFast, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split



In [3]:
def Read_Data(data_dir):
    articles = []
    research_problem_sentences = []

    for category in os.listdir(data_dir):
        if category != 'README.md' and category != '.git':
            article_category = os.path.join(data_dir, category)
            if os.path.isfile(article_category):
                continue

            for folder_name in sorted(os.listdir(article_category)):
                article_index = os.path.join(article_category, folder_name)

                if len(glob.glob(os.path.join(article_index, '*-Stanza-out.txt'))) == 0:
                    continue

                with open(glob.glob(os.path.join(article_index, '*-Stanza-out.txt'))[0], encoding='utf-8') as f:
                    article = f.read()
                    articles.append(article.lower())

                with open(os.path.join(article_index, 'info-units/research-problem.json'), encoding='utf-8') as f:
                    research_problem_sentences.append([element[-1].get("from sentence", None) if isinstance(element, list) else element.get("from sentence", None) if isinstance(element, dict) else None for element in json.load(f)["has research problem"]])
    return articles, research_problem_sentences

In [20]:
!spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 6.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In order to label our dataset to "non research problem" and "research problem", we compare every sentences in research-problem.json with sentences of their related research papaer plain text in stanza format. if the similarity is more that 0.9, label will be 1 ( is research problem sentence) and otherwise, the label will be 0. 
In the bellow cell, we define a function to sompute similarity between two sentences. 

In [69]:
import json , os , glob
import spacy
nlp = spacy.load("en_core_web_lg")

def check_similarity(sentences , train_research_problem_sentences):
    global df
    ########################## Check Similarity

    for i in range(len(train_research_problem_sentences)):
      #print(train_research_problem_sentences[i])
      #print(len(train_research_problem_sentences[i]))
      for j in range(len(train_research_problem_sentences[i])):
        print(train_research_problem_sentences[i][j])
        print("-----------------")
        print(sentences)
        nlp_doc1 = nlp(train_research_problem_sentences[i][j])


      nlp_doc2 = nlp(sentences)

      sim = nlp_doc1.similarity(nlp_doc2)
      print(sim)
      if sim > 0.9:
          '''print("reserch problem : ", nlp_doc1)
          print("")
          print("Sentence :" ,  sentences[i])
          #print(nlp_doc1)
          print("similarity : " , sim)

          print("---------------------------")

          df2 = {'Sentence': sentences[i], 'RP': nlp_doc1, 'prop': sim}
          df = df.append(df2, ignore_index = True)'''
          return 1
      else:
          return 0

In [70]:
'''sentences = []
labels = []
for i, article in enumerate(train_articles):
    for sentence in article.split('\n')[0:-1]:
        sentences.append(sentence)

        labels.append(int(check_similarity(sentence, train_research_problem_sentences)))'''

A Hierarchical Model for Data - to - Text Generation
-----------------
title
Transcribing structured data into natural language descriptions has emerged as a challenging task , referred to as " data - to - text " .
-----------------
title
0.4332305433690145
A Hierarchical Model for Data - to - Text Generation
-----------------
a hierarchical model for data - to - text generation
Transcribing structured data into natural language descriptions has emerged as a challenging task , referred to as " data - to - text " .
-----------------
a hierarchical model for data - to - text generation
0.8598829851373043
A Hierarchical Model for Data - to - Text Generation
-----------------
abstract
Transcribing structured data into natural language descriptions has emerged as a challenging task , referred to as " data - to - text " .
-----------------
abstract
0.4180977772107475
A Hierarchical Model for Data - to - Text Generation
-----------------
transcribing structured data into natural language desc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Streaming output truncated to the last 5000 lines.
on the other hand , training on oracle clusters leads to a mismatch between training and test , which can hurt performance .
0.851674628559633
A Hierarchical Model for Data - to - Text Generation
-----------------
search
Transcribing structured data into natural language descriptions has emerged as a challenging task , referred to as " data - to - text " .
-----------------
search
0.5058796930922321
A Hierarchical Model for Data - to - Text Generation
-----------------
when moving from a strictly local objective to one with global features , the test - time search problem becomes intractable .
Transcribing structured data into natural language descriptions has emerged as a challenging task , referred to as " data - to - text " .
-----------------
when moving from a strictly local objective to one with global features , the test - time search problem becomes intractable .
0.8988668345980498
A Hierarchical Model for Data - to - Text Gene

In [38]:
def create_dataset(articles, research_problem_sentences):
    sentences = []
    labels = []
    for i, article in enumerate(articles):
        for sentence in article.split('\n')[0:-1]:
            sentences.append(sentence)
            for i in range(len(train_research_problem_sentences)):

              labels.append(int(check_similarity(sentence, train_research_problem_sentences)))
    return sentences, labels

In [71]:
train_data_dir = '/content/test-data'

train_articles, train_research_problem_sentences = Read_Data(train_data_dir)

train_sentences, train_labels = create_dataset(train_articles, train_research_problem_sentences)


train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_sentences, train_labels, test_size=.2)


In [72]:
print (set(train_labels))

{0, 1}


#Check Imbalence data
we can see bellow that the dataset is imbalanced and our minority class is 1.
so, in the nex steps we try to figure out it.

In [73]:
from collections import Counter

Counter(train_labels).keys() 
Counter(train_labels).values() 



dict_values([26103, 937])

##Tokenization
In this task i have used "SciBERT" pretrain model wich is a BERT model for scientific text. for both tokenization and training.

In [75]:
tokenizer = BertTokenizerFast.from_pretrained('allenai/scibert_scivocab_uncased')
config = BertConfig.from_pretrained('allenai/scibert_scivocab_uncased')
config.num_labels = 2

train_encodings = tokenizer(train_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_sentences, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
print('Train Data Size:({0})'.format(len(train_labels)))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
print('Validation Data Size:({0})'.format(len(val_labels)))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Train Data Size:(27040)
Validation Data Size:(6760)


In [76]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#Overcome Imbalance Dataset
becuase the dataset is imbalance, we should use class weight in our training process. 
we used class_weight from sklearn to compute the class weight.

In [77]:
import sklearn
from sklearn.utils import class_weight
import numpy as np
class_weights = sklearn.utils.class_weight.compute_class_weight('balanced', classes=np.unique(train_labels),y=train_labels)

In [78]:
class_weights

array([ 0.51794813, 14.42902882])

In [ ]:
weights_dict = {i:w for i,w in enumerate(class_weights)}
EPOCHS = 8
BATCH_SIZE = 8
lr=1e-5

Becuase lack of enough time and time consuming model, I can just train in few epochs and also I can only impelement one approach to find research problem, in the next steps, I will try with detect the span of a candidate research problem sentence and generating research topic for a given research papaer using text generation models like seq2seq models.
this impelementation was just a try to do the task, It will be better with more complex models, using BI-LSTMs and will be optimize.

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', from_pt=True, config=config)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= lr, epsilon=1e-8),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', f1_m, precision_m, recall_m])

history=model.fit(train_dataset.shuffle(len(train_labels)).batch(BATCH_SIZE),
          validation_data=val_dataset.batch(BATCH_SIZE),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight=weights_dict)

model.save_weights('/content/drive/MyDrive/ncg/model-SC-BERT/')
print('*****************model saved!******************')



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/8
3380/3380 [==============================] - 2478s 728ms/step - loss: 0.2800 - accuracy: 0.8561 - f1_m: 0.0524 - precision_m: 0.0312 - recall_m: 0.1871 - val_loss: 0.4579 - val_accuracy: 0.7664 - val_f1_m: 0.0847 - val_precision_m: 0.0533 - val_recall_m: 0.2308
Epoch 2/8
3380/3380 [==============================] - 2456s 727ms/step - loss: 0.1572 - accuracy: 0.9186 - f1_m: 0.0592 - precision_m: 0.0345 - recall_m: 0.2277 - val_loss: 0.3414 - val_accuracy: 0.8967 - val_f1_m: 0.0633 - val_precision_m: 0.0371 - val_recall_m: 0.2396
Epoch 3/8
2797/3380 [=======================>......] - ETA: 6:36 - loss: 0.1201 - accuracy: 0.9398 - f1_m: 0.0596 - precision_m: 0.0347 - recall_m: 0.2314